This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.4) and depends on Magma (tested using version
2.25-5).

In this notebook, we identify non-trigonal curves of genus 5 which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1. Allow 1 hour for completion.

In [57]:
load("preamble.sage")

Construct the $\mathbb{F}_2$-rational points and $\mathbb{F}_4$-rational points of $\mathbf{P}^4$.

In [2]:
F = GF(2)
P.<x0,x1,x2,x3,x4> = F[]

In [3]:
S0 = [vector(t) for t in ProjectiveSpace(F, 4).rational_points()]
for v in S0:
    v.set_immutable()
len(S0)

31

In [4]:
F4 = GF(4)
S04 = [vector(t) for t in ProjectiveSpace(F4, 4).rational_points()]
len(S04)

341

Construct the vector of quadratic monomials in 5 variables and the space of quadrics.

In [5]:
monos2 = [prod(x) for x in itertools.combinations_with_replacement(P.gens(), 2)]
len(monos2)

15

In [6]:
def vec_to_gen(v, monos2=monos2):
    return sum(v[i]*monos2[i] for i in range(15))

In [7]:
def gen_to_vec(gen, monos2=monos2):
    return vector(F, (gen.coefficient(mu) for mu in monos2))

In [8]:
S = []
for v in VectorSpace(F, 15):
    gen = vec_to_gen(v)
    if gen == 0: 
        continue
    fac = gen.factor()
    if len(fac) > 1 or fac[0][1] > 1:
        continue
    pts = [x for x in S0 if gen(*x) == 0]
    if len(pts) in [11, 15, 19]:
        S.append(gen)
len(S)

32116

Implement the action of $\mathrm{GL}(5, \mathbb{F}_2)$ on quadrics.

In [9]:
def apply_group_elem(g, gen):
    return gen(*~g*vector(P.gens()))

Hard-code Dragutinović's representatives for the space of nondegenerate quadrics in 5 variables.

In [10]:
repr = [(1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0), 
        (0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0),
        (0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0), 
        (0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0)]

Hard-code Dragutinović's generators for the stabilizers of these, rewrite using shorter generating sequences, then verify.

In [11]:
Stab1 = [matrix([[0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 1, 0, 1, 1], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 0, 0, 0, 1], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [1, 1, 1, 0, 1], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 1, 1, 0, 1], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 1], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]), matrix([[1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 0, 0, 0, 1], [0, 1, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 0, 0, 0, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1], [1, 0, 0, 0, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1], [1, 0, 0, 0, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 0, 0, 0, 1], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 0, 1, 0, 1], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1], [0, 1, 0, 0, 0]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]), matrix([[1, 1, 1, 0, 1], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 1, 0, 1, 1], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 1, 0, 1, 1], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 0, 1, 0, 1], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 1, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 1], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 1], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 1, 1, 0, 1], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 1, 1, 0, 1], [0, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0]]), matrix([[0, 1, 0, 1, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0]]), matrix([[1, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 1, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 1, 0, 1, 1], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 1, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 1, 1, 0, 1], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 0, 1], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 0, 1], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 1, 0], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 1, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 1, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 0, 1], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 0, 1], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 1, 0], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 1, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 1, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 1, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 1, 0, 1, 1], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 1, 0, 1, 1], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 1, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 1, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 1, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 1, 1, 0, 1], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 1, 1, 0, 1], [0, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 1, 1, 0, 1], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 1, 0], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 1, 0], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 1, 1, 0, 1], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 1], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 1], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 1], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 1], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 1], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 1], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 1], [1, 1, 1, 0, 1], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 1], [0, 0, 1, 0, 1], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 1], [0, 0, 1, 0, 1], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 1], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [1, 0, 0, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 1], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 1], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [1, 1, 1, 0, 1], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 1], [0, 0, 0, 0, 1], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [0, 0, 0, 0, 1], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [0, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 0, 1], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [0, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 1], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 0, 0, 0, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 0, 0, 0, 1], [1, 1, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1], [1, 0, 0, 0, 1], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1], [1, 0, 0, 0, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1], [1, 1, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [0, 0, 1, 0, 1], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 1], [0, 0, 0, 0, 1], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 0, 0, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 1, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 1, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 1], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 0, 0, 1], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 0, 0, 0, 1], [1, 1, 1, 0, 1], [1, 0, 0, 0, 1], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 1], [1, 1, 1, 0, 1], [1, 0, 0, 0, 1], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 1, 0, 0, 1], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 1], [0, 0, 1, 0, 1], [0, 1, 0, 0, 1], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 1], [0, 0, 0, 0, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1]]), matrix([[0, 0, 0, 0, 1], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 1], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1]]), matrix([[0, 0, 0, 0, 1], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 1], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 1], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 1], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1]])]

Stab3 = [matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]])]

Stab4 = [matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 1, 0]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 1]])]

Stab6 = [matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 0, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [0, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 0, 0, 0, 0], [0, 0, 1, 1, 0], [1, 1, 0, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 0, 0, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 0, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [1, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[0, 0, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]]), matrix([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 0, 1, 1, 0], [1, 1, 1, 1, 1]])]

In [12]:
G = GL(5, F)
stab = [G.subgroup(l) for l in [Stab1, Stab3, Stab4, Stab6]]
stab = [G.subgroup([g.matrix() for g in random_generating_sequence(i)]) for i in stab]
[len(i.gens()) for i in stab]

[3, 4, 10, 8]

In [13]:
for i in range(4):
    gen = vec_to_gen(repr[i])
    for g in stab[i]:
        g1 = Matrix(F, g)
        gen2 = apply_group_elem(g1, gen)
        assert gen == gen2

Construct an orbit lookup tree to depth 1 on nondegenerate quadrics. This does not require the stabilizer function.

In [14]:
def optimized_rep(g):
    return g.matrix()

In [15]:
methods = {
    'apply_group_elem': apply_group_elem,
    'optimized_rep': optimized_rep,
    'stabilizer': None
}
tree = {}
G = GL(5, F)
extend_orbit_tree(G, S, tree, methods, verbose=True)

Current level: 0
Number of generators: 2
Edges computed
Retract computed
Number of new nodes: 4
Edges computed
Retract computed
Stabilizer generators found
Number of new green nodes: 4
New level: 1


Create a lookup table to indicate which element belongs to which orbit.

In [16]:
tree_repr = green_nodes(tree, 1)
tree_lookup = {}
for gen in S:
    rep, g = orbit_rep_from_tree(G, tree, (gen,), apply_group_elem, optimized_rep)
    tree_lookup[gen] = tree_repr.index(rep)

Correlate Dragutinovic's orbit representatives to ours.

In [17]:
repr_conv = [orbit_rep_from_tree(G, tree, (vec_to_gen(repr[i]),), apply_group_elem, optimized_rep) for i in range(4)]

Construct stabilizers by transferring Dragutinovic's matrices, then check the result.

In [18]:
def stabilizer(gen, G=G, tree=tree, repr_conv=repr_conv, 
               apply_group_elem=apply_group_elem, optimized_rep=optimized_rep):
    tmp = orbit_rep_from_tree(G, tree, (gen,), apply_group_elem, optimized_rep)
    tmp2 = [j for j in range(4) if repr_conv[j][0] == tmp[0]]
    assert len(tmp2) == 1
    j = tmp2[0]
    g = repr_conv[j][1]*~tmp[1]
    return G.subgroup([~g*Matrix(F, m)*g for m in stab[j]])

In [19]:
for (gen,) in green_nodes(tree, 1):
    tmp = orbit_rep_from_tree(G, tree, (gen,), apply_group_elem, optimized_rep)
    assert apply_group_elem(tmp[1], tmp[0][0]) == gen
    assert all(apply_group_elem(g, gen) == gen for g in stabilizer(gen).gens())

Extend the orbit tree to depth 2. We forbid pairs in which the third member of the pencil is not nondegenerate.

In [20]:
def forbid(gens, easy=False, tree_lookup=tree_lookup):
    if len(gens) == 2 and gens[0] + gens[1] not in tree_lookup:
        return True
    return False

In [21]:
methods['stabilizer'] = stabilizer
methods['forbid'] = forbid

In [22]:
extend_orbit_tree(G, S, tree, methods, verbose=True)

Current level: 1
Number of generators: 3
Edges computed
Retract computed
Number of generators: 3
Edges computed
Retract computed
Number of generators: 3
Edges computed
Retract computed
Number of generators: 3
Edges computed
Retract computed
Number of new nodes: 483
Edges computed
Retract computed
Stabilizer generators found
Number of new green nodes: 228
New level: 2


We may impose an order condition on the orbits of the generators without loss of generality. Pick out orbit representatives meeting this condition.

In [23]:
good_gens = [gens for gens in green_nodes(tree, 2) if tree_lookup[gens[0]] <= tree_lookup[gens[1]] and \
                                                        tree_lookup[gens[1]] <= tree_lookup[gens[0]+gens[1]]]
len(good_gens)

112

For each pair of quadrics (with the second in an equal or later orbit than the first) and each subset $T$ of their common zero set of size in $\{2,\dots,6\}$, solve for a third quadric such that the triple intersection has $\mathbb{F}_2$-rational points equal to $T$. We also enforce the condition that all of the quadrics in the pencil are nondegenerate, impose the WLOG order condition on the orbits, and check the point count over $\mathbb{F}_4$.

In [24]:
def redundancy(vecs):
    return Matrix(vecs)

In [25]:
coords = {x: vector(F, (mu(*x) for mu in monos2)) for x in S0}
coords4 = {x: vector(F4, (mu(*x) for mu in monos2)) for x in S04}

In [26]:
curves = defaultdict(list)
tmp2 = [t[:2] for t in targets5]
for gens in good_gens:
    vecs = [gen_to_vec(gen) for gen in gens]
    pts = [x for x in S0 if all(gen(*x) == 0 for gen in gens)]
    pts2 = [x for x in S04 if all(gen(*x) == 0 for gen in gens)]
    perp = Matrix([coords[x] for x in pts])
    for s in range(2, 7):
        for T in itertools.combinations(pts, s):
            target = vector(F, (0 if x in T else 1 for x in pts))
            for w in solve_right_iterator(perp, target, redundancy, vecs):
                gen3 = vec_to_gen(w)
                if any(gen3+i0*gens[0]+i1*gens[1] not in tree_lookup or \
                    tree_lookup[gens[1]] > tree_lookup[gen3+i0*gens[0]+i1*gens[1]] \
                    for i0 in range(2) for i1 in range(2)):
                    continue
                s2 = sum(1 for x in pts2 if gen3(*x) == 0)
                if (s,s2) in tmp2:
                    curves[(s,s2)].append(gens + (gen3,))

In [27]:
[(s, len(curves[s])) for s in curves]

[((2, 10), 3418),
 ((3, 11), 4539),
 ((3, 9), 9311),
 ((4, 8), 15275),
 ((4, 10), 8385),
 ((4, 12), 3860),
 ((4, 14), 1246),
 ((5, 11), 6683),
 ((5, 9), 9815),
 ((5, 13), 2394),
 ((6, 12), 4038),
 ((6, 10), 4819)]

Construct curves from the resulting triples of generators, then impose conditions on the point counts over $\mathbb{F}_{2^i}$ for $i=3,4$.

In [28]:
def count_by_ideal(gens, n):
    J = P.ideal(list(gens) + [y^(2^n) + y for y in P.gens()])
    return (J.vector_space_dimension() - 1) // (2^n-1)

In [29]:
for n in range(3, 5):
    tmp = [t[:n] for t in targets5]
    tmp2 = list(curves.keys())
    for s in tmp2:
        if len(s) == n-1:
            for gen in curves[s]:
                i = count_by_ideal(gen, n)
                s1 = s + (i,)
                if s1 in tmp:
                    curves[s1].append(gen)
    print([(s, len(curves[s])) for s in curves if len(s) == n])

[((2, 10, 20), 33), ((3, 11, 15), 673), ((3, 11, 9), 1404), ((3, 9, 18), 436), ((4, 8, 16), 3408), ((4, 10, 16), 1464), ((4, 10, 10), 2084), ((4, 12, 16), 621), ((4, 12, 4), 294), ((4, 12, 10), 902), ((4, 14, 10), 451), ((5, 11, 17), 1492), ((5, 11, 11), 1468), ((5, 11, 5), 278), ((5, 9, 17), 2499), ((5, 9, 14), 1693), ((5, 9, 11), 1866), ((5, 9, 8), 1450), ((5, 9, 5), 803), ((5, 13, 14), 684), ((5, 13, 8), 144), ((5, 13, 5), 275), ((6, 12, 12), 598), ((6, 10, 15), 641), ((6, 10, 12), 801)]
[((2, 10, 20, 18), 8), ((3, 11, 15, 35), 12), ((3, 9, 18, 41), 133), ((3, 9, 18, 25), 26), ((4, 8, 16, 32), 1683), ((4, 10, 16, 26), 79), ((4, 10, 10, 26), 238), ((4, 10, 10, 34), 71), ((4, 12, 16, 24), 302), ((4, 12, 4, 24), 60), ((4, 12, 4, 32), 13), ((4, 12, 10, 24), 185), ((4, 12, 10, 32), 51), ((4, 14, 10, 26), 30), ((5, 11, 11, 27), 118), ((5, 11, 11, 19), 33), ((5, 11, 5, 19), 115), ((5, 11, 5, 27), 4), ((5, 9, 17, 25), 34), ((5, 9, 14, 33), 661), ((5, 9, 11, 25), 168), ((5, 9, 11, 33), 71), 

Use Magma to compute the full zeta functions of the resulting curves, and compare these to our targets.

In [30]:
l = []
proj = magma.ProjectiveSpace(P)
for s in curves:
    if len(s) == 4:
        for gens in curves[s]:
            X = proj.Scheme(gens)
            if X.Dimension() != 1 or str(X.IsIrreducible()) == "false" or str(X.IsNonsingular()) == "false":
                continue
            C = X.Curve()
#            assert C.Genus() == 5
            ct = tuple(Integer(C.NumberOfPlacesOfDegreeOneECF(i)) for i in range(1, 6))
            assert ct[:4] == s
            if ct in targets5:
                l.append(C)
len(l)

524

Finish this case.

In [ ]:
closeout(l)